In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')
from src.models.model import HistoricalMedian,XGBoost,LinearModel,RF,KNN,SVM,mase,TimeSeriesSplitImproved
from src.data.split_all_data import DROPCOLS_DIFF_FINAL,DROPCOLS_FINAL
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler,Normalizer,StandardScaler,QuantileTransformer
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split,cross_val_predict
from skgarden.quantile import RandomForestQuantileRegressor
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error
from sklearn.preprocessing import FunctionTransformer,PolynomialFeatures
from sklearn.svm import SVR
from copy import copy
from tpot.builtins import StackingEstimator
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, make_union
from catboost import CatBoostRegressor,Pool,cv


df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh_6.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')


Using TensorFlow backend.


In [2]:
year = 2019
tgt = 'rougher.output.recovery'
#cols_drop = X.columns[X.columns.str.contains('p24')]
X = data_dict[year][f'X_train_tsclean']#.drop(cols_drop,axis = 1)
y = data_dict[year]['y_train']
mask = data_dict[year]['mask']

X =X.loc[mask.index,:][mask]
y = y.loc[mask.index,:][mask]
print(f'2)  train: {X.shape}')
#X = X.sample(frac=0.4,random_state=123).sort_index().dropna().astype(float)
y= y.loc[X.index,tgt]
X_filt = X.filter(regex  ="rougher|dayw|hour",axis = 1)
print(f'after sample() train: {X.shape}')
#print(f'after sample() filt train: {X_filt.shape}')
# Nmonths_total = 8
# Nspl = int(Nmonths_total * 30 / 15)
# Nmonths_test = 4
# Nmonths_min_train = 2.5
# train_splits = Nspl // Nmonths_total*Nmonths_min_train
# test_splits=int(Nmonths_test / Nmonths_total*Nspl)
# cv = TimeSeriesSplitImproved(n_splits=Nspl)

2)  train: (13590, 763)
after sample() train: (13590, 763)


# CV functions

In [3]:
n_folds = 5


def cv_boost(model,X_base=None,y=None):
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)
    train_index = X_base.index.year.isin([2016,2017])
    valid_index = X_base.index.year.isin([2018])
    
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    evalset = [(X_valid.values, y_valid.values.reshape(-1,))]
    model.fit(X_train.values,y_train.values.reshape(-1,))
        
    #ntree = model.best_iteration_
    preds  = model.predict(X_valid.values)
        
        
    preds_all_base[valid_index] = preds
    true_all[valid_index] = y_valid
        
    oof_scores = mase(preds_all_base,true_all)
    return oof_scores

def cv_svr(model,X_base=None,y=None,scaler=None):
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)
    train_index = X_base.index.year.isin([2016,2017])
    valid_index = X_base.index.year.isin([2018])
    
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    X_train,X_valid = scaler.transform(X_train),scaler.transform(X_valid)
    model.fit(X_train.values,y_train.values.reshape(-1,))
        
    #ntree = model.best_iteration_
    preds  = model.predict(X_valid.values)
        
        
    preds_all_base[valid_index] = preds
    true_all[valid_index] = y_valid
        
    oof_scores = mean_absolute_error(preds_all_base,true_all)
    return oof_scores

def cv_lgboost(model,X_base=None,y=None):
    scores = []
    preds_all_alt = np.empty_like(y)
    preds_all_base = np.empty_like(y)
    
    true_all =np.empty_like(y)
    train_index = X_base.index.year.isin([2016,2017])
    valid_index = X_base.index.year.isin([2018])
    
    X_train, X_valid = X_base.iloc[train_index], X_base.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    model.fit(X_train.values,y_train.values.reshape(-1,))
        
    #ntree = model.best_iteration_
    preds  = model.predict(X_valid.values)
        
        
    oof_scores = mase(preds,y_valid.values)
    return oof_scores



def cv_oof(model,X_base=None,y=None):
    cv = KFold(n_folds, shuffle=False, random_state=42)
    preds = cross_val_predict(model,X=X_base,y=y,cv=cv,n_jobs = 1,method = 'predict')
    oof_scores = mase(preds,y)
    return oof_scores


# Try hyperopt for LGBM

In [8]:
N =100
feature_subset_df = pd.read_csv('./shap-importance-rougher-diff_deriv.csv')
feature_subset = feature_subset_df['feature'].head(N)

X_sub=X.copy()
X_sub.columns = [x.replace("\"","") for x in X_sub.columns]

In [6]:
from hyperopt import hp, tpe,Trials
from hyperopt.fmin import fmin
X_sub = X_sub[feature_subset]

def objective(params):
    params = {
        #'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']), #int(max(2**(int(params['max_depth'])) - params['num_leaves'],0)),
        'feature_fraction': "{:.3f}".format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction']),
        'lambda_l1':params['lambda_l1']
        #"min_data_in_leaf": int(params['min_data_in_leaf'])

    }
    
    m = lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=900,random_state=9, 
                              **params)
    sc = cv_lgboost(m,X_base = X_sub,y=y)
    print("Score {:.3f} params {}".format(sc, params))
    return sc

space = {
   # 'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'num_leaves': hp.quniform('num_leaves', 8, 50, 1 ),
    'feature_fraction': hp.uniform('feature_fraction', 0.02, 0.9),
    'bagging_fraction':hp.uniform('bagging_fraction', 0.1, 1.0),
    'lambda_l1':hp.uniform('lambda_l1',0.1,80)
    #"min_data_in_leaf": hp.loguniform("min_data_in_leaf",1,8)
}
trials = Trials()
best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150,trials=trials)
# For Rougher, tsclean
# Score 2.208 params {'num_leaves': 21, 'feature_fraction': '0.664', 'bagging_fraction': '0.453', 'lambda_l1': 7.1020592954595525}  -- MAE
# Score 2.206 params {'num_leaves': 25, 'feature_fraction': '0.183', 'bagging_fraction': '0.410', 'lambda_l1': 2.3256438279612306}
# Score 2.229 params {'num_leaves': 23, 'feature_fraction': '0.710', 'bagging_fraction': '0.285', 'lambda_l1': 0.16223732523759438}
# Score 2.281 params {'num_leaves': 24, 'feature_fraction': '0.185', 'bagging_fraction': '0.314', 'lambda_l1': 34.88656438035975}
# Score 2.214 params {'num_leaves': 23, 'feature_fraction': '0.202', 'bagging_fraction': '0.365', 'lambda_l1': 3.9789175088379785}
# Score 2.178 params {'num_leaves': 50, 'feature_fraction': '0.679', 'bagging_fraction': '0.436', 'lambda_l1': 6.657040324436044}
# Score 2.298 params {'num_leaves': 45, 'feature_fraction': '0.482', 'bagging_fraction': '0.860', 'lambda_l1': 24.954518604134595}
# Score 2.225 params {'num_leaves': 19, 'feature_fraction': '0.841', 'bagging_fraction': '0.756', 'lambda_l1': 39.0534423460024}
# Score 2.213 params {'num_leaves': 47, 'feature_fraction': '0.822', 'bagging_fraction': '0.820', 'lambda_l1': 39.31828209933644}
# Score 2.170 params {'num_leaves': 34, 'feature_fraction': '0.706', 'bagging_fraction': '0.596', 'lambda_l1': 34.544882299094574}
# Score 2.194 params {'num_leaves': 28, 'feature_fraction': '0.888', 'bagging_fraction': '0.456', 'lambda_l1': 32.76795766388388}
# Score 2.148 params {'num_leaves': 15, 'feature_fraction': '0.865', 'bagging_fraction': '0.595', 'lambda_l1': 13.053656812210047}
# Score 2.156 params {'num_leaves': 29, 'feature_fraction': '0.860', 'bagging_fraction': '0.502', 'lambda_l1': 21.35360888869841}


# For final, tsclean:
# Score 2.111 params {'num_leaves': 30, 'feature_fraction': '0.639', 'bagging_fraction': '0.564', 'lambda_l1': 30.404618090697902}
# Score 2.124 params {'num_leaves': 32, 'feature_fraction': '0.417', 'bagging_fraction': '0.130', 'lambda_l1': 26.335647951629017}
# Score 2.058 params {'num_leaves': 41, 'feature_fraction': '0.427', 'bagging_fraction': '0.817', 'lambda_l1': 25.80144044677823}
# Score 2.025 params {'num_leaves': 46, 'feature_fraction': '0.367', 'bagging_fraction': '0.675', 'lambda_l1': 16.92997392193761}
# Score 2.037 params {'num_leaves': 28, 'feature_fraction': '0.509', 'bagging_fraction': '0.332', 'lambda_l1': 20.66766776240585}
# Score 2.084 params {'num_leaves': 20, 'feature_fraction': '0.512', 'bagging_fraction': '0.620', 'lambda_l1': 20.6878922166273}
# 600 features:
# Score 1.992 params {'num_leaves': 45, 'feature_fraction': '0.308', 'bagging_fraction': '0.129', 'lambda_l1': 41.339064272539595}
# Score 1.992 params {'num_leaves': 42, 'feature_fraction': '0.526', 'bagging_fraction': '0.603', 'lambda_l1': 53.242352401598005}
# Score 2.007 params {'num_leaves': 10, 'feature_fraction': '0.269', 'bagging_fraction': '0.315', 'lambda_l1': 77.05034951120668}
# Score 1.993 params {'num_leaves': 14, 'feature_fraction': '0.144', 'bagging_fraction': '0.832', 'lambda_l1': 62.74777174328949}
# Score 2.000 params {'num_leaves': 11, 'feature_fraction': '0.580', 'bagging_fraction': '0.256', 'lambda_l1': 7.765530266350743}
# Score 1.955 params {'num_leaves': 23, 'feature_fraction': '0.211', 'bagging_fraction': '0.285', 'lambda_l1': 57.77419482071672}
# Score 1.973 params {'num_leaves': 25, 'feature_fraction': '0.187', 'bagging_fraction': '0.385', 'lambda_l1': 50.931545559984954}
# Score 1.941 params {'num_leaves': 18, 'feature_fraction': '0.180', 'bagging_fraction': '0.353', 'lambda_l1': 48.949906681045405}
# Score 1.952 params {'num_leaves': 9, 'feature_fraction': '0.253', 'bagging_fraction': '0.537', 'lambda_l1': 37.93719014242221}
# Score 1.932 params {'num_leaves': 13, 'feature_fraction': '0.179', 'bagging_fraction': '0.393', 'lambda_l1': 30.378882925726465}
# Score 1.895 params {'num_leaves': 11, 'feature_fraction': '0.200', 'bagging_fraction': '0.259', 'lambda_l1': 31.046267749256195}
# 500 features:
# Score 1.951 params {'num_leaves': 21, 'feature_fraction': '0.190', 'bagging_fraction': '0.594', 'lambda_l1': 38.619965101613104}
# Score 1.945 params {'num_leaves': 20, 'feature_fraction': '0.182', 'bagging_fraction': '0.683', 'lambda_l1': 46.634309291667165}
# Score 1.955 params {'num_leaves': 11, 'feature_fraction': '0.420', 'bagging_fraction': '0.459', 'lambda_l1': 35.03363039232508}
# Score 1.961 params {'num_leaves': 28, 'feature_fraction': '0.421', 'bagging_fraction': '0.479', 'lambda_l1': 47.41477249840837}
 # Score 1.950 params {'num_leaves': 19, 'feature_fraction': '0.170', 'bagging_fraction': '0.449', 'lambda_l1': 17.385879957071193}
# Score 1.907 params {'num_leaves': 12, 'feature_fraction': '0.162', 'bagging_fraction': '0.618', 'lambda_l1': 79.69691894954272}
 # Score 1.932 params {'num_leaves': 10, 'feature_fraction': '0.203', 'bagging_fraction': '0.644', 'lambda_l1': 73.12030419750744}
# Score 1.907 params {'num_leaves': 8, 'feature_fraction': '0.195', 'bagging_fraction': '0.710', 'lambda_l1': 66.45449792425866}
# Score 1.924 params {'num_leaves': 11, 'feature_fraction': '0.202', 'bagging_fraction': '0.706', 'lambda_l1': 79.82409578329322}
# 400 features
# Score 1.960 params {'num_leaves': 15, 'feature_fraction': '0.162', 'bagging_fraction': '0.359', 'lambda_l1': 65.8303050417742}
# Score 1.961 params {'num_leaves': 26, 'feature_fraction': '0.416', 'bagging_fraction': '0.700', 'lambda_l1': 11.42976771620764}
# Score 1.921 params {'num_leaves': 14, 'feature_fraction': '0.054', 'bagging_fraction': '0.452', 'lambda_l1': 14.05424920482756}
# Score 1.955 params {'num_leaves': 46, 'feature_fraction': '0.156', 'bagging_fraction': '0.549', 'lambda_l1': 24.79520895384721}
# Score 1.954 params {'num_leaves': 33, 'feature_fraction': '0.677', 'bagging_fraction': '0.875', 'lambda_l1': 30.94461286745046}
# Score 1.924 params {'num_leaves': 18, 'feature_fraction': '0.786', 'bagging_fraction': '0.739', 'lambda_l1': 38.263422120356964}
# Score 1.935 params {'num_leaves': 13, 'feature_fraction': '0.315', 'bagging_fraction': '0.336', 'lambda_l1': 49.30365642603196}

# 300 features
# Score 1.930 params {'num_leaves': 11, 'feature_fraction': '0.114', 'bagging_fraction': '0.692', 'lambda_l1': 18.444918611299318}
# Score 1.967 params {'num_leaves': 15, 'feature_fraction': '0.377', 'bagging_fraction': '0.392', 'lambda_l1': 34.07301147862394}
# Score 1.950 params {'num_leaves': 41, 'feature_fraction': '0.585', 'bagging_fraction': '0.741', 'lambda_l1': 20.75987814974435}
# Score 1.948 params {'num_leaves': 12, 'feature_fraction': '0.544', 'bagging_fraction': '0.329', 'lambda_l1': 13.845810268500632}
# Score 1.939 params {'num_leaves': 8, 'feature_fraction': '0.305', 'bagging_fraction': '0.260', 'lambda_l1': 61.79123147378411}
# Score 1.932 params {'num_leaves': 16, 'feature_fraction': '0.178', 'bagging_fraction': '0.341', 'lambda_l1': 51.80532249201323}
# Score 1.922 params {'num_leaves': 43, 'feature_fraction': '0.805', 'bagging_fraction': '0.938', 'lambda_l1': 23.770769532296903}
# Score 1.912 params {'num_leaves': 35, 'feature_fraction': '0.558', 'bagging_fraction': '0.663', 'lambda_l1': 35.2515557353255}
# Score 1.951 params {'num_leaves': 27, 'feature_fraction': '0.596', 'bagging_fraction': '0.799', 'lambda_l1': 16.298917308077932}

# 200 features
# Score 1.878 params {'num_leaves': 34, 'feature_fraction': '0.197', 'bagging_fraction': '0.448', 'lambda_l1': 59.99267076535807}
# Score 1.890 params {'num_leaves': 25, 'feature_fraction': '0.124', 'bagging_fraction': '0.314', 'lambda_l1': 53.16005608163073}
# Score 1.878 params {'num_leaves': 34, 'feature_fraction': '0.197', 'bagging_fraction': '0.448', 'lambda_l1': 59.99267076535807}
# Score 1.871 params {'num_leaves': 17, 'feature_fraction': '0.557', 'bagging_fraction': '0.801', 'lambda_l1': 6.952195513599493}
# Score 1.860 params {'num_leaves': 20, 'feature_fraction': '0.087', 'bagging_fraction': '0.621', 'lambda_l1': 30.352952423993347}
# 100 features
# Score 1.805 params {'num_leaves': 30, 'feature_fraction': '0.108', 'bagging_fraction': '0.308', 'lambda_l1': 18.385076125138344}
# Score 1.850 params {'num_leaves': 21, 'feature_fraction': '0.485', 'bagging_fraction': '0.143', 'lambda_l1': 32.58364182920104}
# Score 1.806 params {'num_leaves': 15, 'feature_fraction': '0.197', 'bagging_fraction': '0.424', 'lambda_l1': 73.37001532056144}
# Score 1.768 params {'num_leaves': 22, 'feature_fraction': '0.052', 'bagging_fraction': '0.214', 'lambda_l1': 14.284200904632039}
# Score 1.759 params {'num_leaves': 18, 'feature_fraction': '0.068', 'bagging_fraction': '0.165', 'lambda_l1': 36.10693410637714}
# Score 1.765 params {'num_leaves': 32, 'feature_fraction': '0.093', 'bagging_fraction': '0.271', 'lambda_l1': 37.92426594558535}
# Score 1.713 params {'num_leaves': 18, 'feature_fraction': '0.072', 'bagging_fraction': '0.153', 'lambda_l1': 34.74834583414519}
# Score 1.739 params {'num_leaves': 44, 'feature_fraction': '0.028', 'bagging_fraction': '0.311', 'lambda_l1': 44.56846871250107}
# Score 1.757 params {'num_leaves': 50, 'feature_fraction': '0.022', 'bagging_fraction': '0.170', 'lambda_l1': 69.22125599448864}
# Score 1.736 params {'num_leaves': 48, 'feature_fraction': '0.024', 'bagging_fraction': '0.331', 'lambda_l1': 54.6313749177543}
# 50 features
# Score 1.736 params {'num_leaves': 48, 'feature_fraction': '0.024', 'bagging_fraction': '0.331', 'lambda_l1': 54.6313749177543}
# Score 1.745 params {'num_leaves': 11, 'feature_fraction': '0.268', 'bagging_fraction': '0.895', 'lambda_l1': 14.040381291391842}
# Score 1.736 params {'num_leaves': 48, 'feature_fraction': '0.024', 'bagging_fraction': '0.331', 'lambda_l1': 54.6313749177543}


Score 5.451 params {'num_leaves': 47, 'feature_fraction': '0.062', 'bagging_fraction': '0.725', 'lambda_l1': 28.10441543537739}
Score 5.510 params {'num_leaves': 29, 'feature_fraction': '0.725', 'bagging_fraction': '0.537', 'lambda_l1': 15.067554474804961}
Score 5.441 params {'num_leaves': 31, 'feature_fraction': '0.285', 'bagging_fraction': '0.563', 'lambda_l1': 10.529939569548567}
Score 5.541 params {'num_leaves': 20, 'feature_fraction': '0.374', 'bagging_fraction': '0.804', 'lambda_l1': 15.589611655903568}
Score 5.331 params {'num_leaves': 40, 'feature_fraction': '0.594', 'bagging_fraction': '0.101', 'lambda_l1': 66.20547174077666}
Score 5.609 params {'num_leaves': 12, 'feature_fraction': '0.213', 'bagging_fraction': '0.509', 'lambda_l1': 48.28074452634562}
Score 5.364 params {'num_leaves': 45, 'feature_fraction': '0.506', 'bagging_fraction': '0.748', 'lambda_l1': 59.092738491729136}
Score 5.533 params {'num_leaves': 49, 'feature_fraction': '0.813', 'bagging_fraction': '0.196', 'lam

KeyboardInterrupt: 

In [ ]:
losses = [trials.trials[i]['result']['loss'] for i in range(len(trials.trials))]
params = pd.DataFrame(trials.vals)
params['loss'] = losses
#pd.DataFrame(trials.vals)
params.sort_values('loss')


In [17]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
def fair_obj(preds, dtrain):
    """y = c * abs(x) - c**2 * np.log(abs(x)/c + 1)"""
    x = preds - dtrain.get_labels()
    c = 1
    den = abs(x) + c
    grad = c*x / den
    hess = c*c / den ** 2
    return grad, hess

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'subsample':"{:.3f}".format(params['subsample']),
        'min_child_weight':"{:.3f}".format(params['min_child_weight']),
        'alpha':params['alpha'],
        'lambda':params['lambda']
        

    }
    # Score 1.226 params {'max_depth': 2, 'gamma': '0.544', 'colsample_bytree': '0.684', 'subsample': '0.932'}
    
    m = xgb.XGBRegressor(learning_rate=0.05,obj = fair_obj,
                             n_estimators=900,                        
                             random_state =123,verbose=0,silent=True, n_jobs = -1,**params)
    score = cv_lgboost(m,X_base = X_sub,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 2, 4, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.05,1.0),
    'min_child_weight':hp.loguniform('min_child_weight', -3, 5),
    'subsample':hp.uniform('subsample', 0.3,1.0),
    'gamma': hp.uniform('gamma',1, 100.),
    'alpha': hp.uniform('alpha',1, 50.),
    'lambda': hp.uniform('lambda',0, 50.)
    
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=250)
#depth 9 gam 7.6 col 0.5



Score 5.307 params {'max_depth': 3, 'gamma': '85.211', 'colsample_bytree': '0.801', 'subsample': '0.785', 'min_child_weight': '15.734', 'alpha': 46.99379559825416, 'lambda': 45.32399625605786}
Score 5.353 params {'max_depth': 3, 'gamma': '75.234', 'colsample_bytree': '0.825', 'subsample': '0.742', 'min_child_weight': '7.478', 'alpha': 25.607946405631242, 'lambda': 36.70746436972148}
Score 5.304 params {'max_depth': 4, 'gamma': '64.180', 'colsample_bytree': '0.096', 'subsample': '0.844', 'min_child_weight': '1.163', 'alpha': 16.060434406701564, 'lambda': 45.23661413465363}
Score 5.504 params {'max_depth': 3, 'gamma': '39.401', 'colsample_bytree': '0.071', 'subsample': '0.367', 'min_child_weight': '4.667', 'alpha': 6.417849315322353, 'lambda': 1.7868431255871897}
Score 5.230 params {'max_depth': 3, 'gamma': '73.318', 'colsample_bytree': '0.927', 'subsample': '0.820', 'min_child_weight': '0.082', 'alpha': 46.511342678703606, 'lambda': 26.37602490596195}
Score 5.342 params {'max_depth': 3,

KeyboardInterrupt: 

In [ ]:
X

In [ ]:
m = LGBMRegressor()
#Xfilt lagged, one lag:
# Score 2.480 params {'max_depth': 5, 'num_leaves': 40, 'feature_fraction': '0.453', 'bagging_fraction': '0.276'}
# Score 2.500 params {'max_depth': 4, 'num_leaves': 180, 'feature_fraction': '0.322', 'bagging_fraction': '0.150'}
# Score 2.490 params {'max_depth': 5, 'num_leaves': 80, 'feature_fraction': '0.440', 'bagging_fraction': '0.666'}
# Score 2.490 params {'max_depth': 6, 'num_leaves': 25, 'feature_fraction': '0.555', 'bagging_fraction': '0.257'}
# Score 2.484 params {'max_depth': 4, 'num_leaves': 30, 'feature_fraction': '0.658', 'bagging_fraction': '0.863'}


# Try HyperOpt for SVR

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
#scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),StandardScaler(),PCA(whiten=True))
scaler = RobustScaler() 
#X_sc = pd.DataFrame(data = scaler.fit_transform(X), index= X.index)
def objective(params):
    params = {
        'C': params['C'],
      #  'epsilon':"{:.3f}".format(params['num_leaves']),
        'gamma': params['gamma']
    }
    
    m = make_pipeline(scaler,SVR(kernel = 'rbf',**params))
    score = cv_boost(m,X_base = X,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'C': hp.loguniform('C', -2, 10),
    'gamma': hp.loguniform('gamma', -2,4 )
}

best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)
#Score 2.560 params {'C': 15.130365120277071, 'gamma': 51.040699174724814}


In [ ]:
cv_boost()

# Try HyperOpt for Catboost

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'l2_leaf_reg': params['l2_leaf_reg'],
       # 'colsample_bylevel': params['colsample_bylevel'],
     #   'random_strength':params['random_strength'],
        #'verbose_eval':1000
    }
    # Score 1.226 params {'max_depth': 2, 'gamma': '0.544', 'colsample_bytree': '0.684', 'subsample': '0.932'}
    m = CatBoostRegressor(loss_function='Quantile:alpha=0.6',random_seed =123,learning_rate=0.4,task_type='GPU',od_type = 'Iter',od_wait= 15,iterations = 1000,
                          max_bin=128,**params)

    score = cv_catboost(m,X_base = X_filt,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 3, 5, 1),
    #'colsample_bylevel': hp.uniform('colsample_bylevel', 0.3, 1.0),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg', -8, 1),
    #'random_strength':hp.loguniform('random_strength', -5, 5)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)

# Try hyperopt KNN

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.decomposition import PCA

scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),StandardScaler(),PCA(whiten=True))
X_sc = scaler.fit_transform(X)
    


def objective(params):
    params = {
        'n_neighbors': int(params['n_neighbors']),
        'weights':params['weights'],
        'leaf_size': params['leaf_size']
    }
    
    
    m = KNeighborsRegressor(n_jobs = -1,**params)
    score = cv_oof(m,X_base = X_sc,y=y)
    print("Score {:.3f} params {}".format(score, params))
    return score

space = {
    'n_neighbors': hp.quniform('n_neighbors',3, 256,2),
    'leaf_size': hp.quniform('lef_size', 14,50,2 ),
    'weights':hp.choice('weights', ['uniform','distance'])
}

best_lgbm = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150)
# Score 3.224 params {'n_neighbors': 198, 'weights': 'distance', 'leaf_size': 36.0}
# Score 3.194 params {'n_neighbors': 254, 'weights': 'distance', 'leaf_size': 16.0}
# Score 3.196 params {'n_neighbors': 250, 'weights': 'distance', 'leaf_size': 18.0}
# Score 3.202 params {'n_neighbors': 238, 'weights': 'distance', 'leaf_size': 48.0}
# Score 3.215 params {'n_neighbors': 214, 'weights': 'distance', 'leaf_size': 30.0}



In [ ]:
y.plot(figsize  = (20,5))

In [ ]:
y.apply(pd.isna,axis=0).sum()

In [ ]:
diffs.plot()

In [ ]:
y[tgt].values